# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [83]:
#imports 
import pandas as pd
import emoji
import emoji_list


In [84]:
#Leitura dos arquivos em Excel

nubank_treinamento = pd.read_excel('tweets_treinamento_p2.xlsx')
nu_treino_sim = nubank_treinamento[nubank_treinamento['Avaliacao'] == 'sim']
nu_treino_nao = nubank_treinamento[nubank_treinamento['Avaliacao'] == 'nao']
nubank_teste = pd.read_excel('tweets_teste_p2.xlsx')

In [85]:
def limpar_caracteres(tabela, titulo):
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t']
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            a = a.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    for e in itens:
        a = a.str.replace(e,'')
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@',' @')
        
    tabela[titulo] = a

    return tabela

def split_tweet(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['palavras'] = []

    for lista in a:
        for e in ['#','' , ' ']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if '@' in palavra and palavra != '@nubank':
                lista.remove(palavra) 
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'palavras': lista}, ignore_index=True)
                
    return nova

def tabela_palavras(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras']

def tabela_tudo(tabela1, tabela2):
    a = tabela1['palavras']
    b = tabela2['palavras']
    tudo = pd.DataFrame()
    tudo['palavras'] = []
    for lista in a:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    for lista in b:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    
    return tudo

In [86]:
palavras_sim = tabela_palavras(split_tweet(limpar_caracteres(nu_treino_sim, 'Treinamento'), 'Treinamento'))
num_sim = palavras_sim.value_counts()
len_sim = len(num_sim)

palavras_nao = tabela_palavras(split_tweet(limpar_caracteres(nu_treino_nao, 'Treinamento'), 'Treinamento'))
num_nao = palavras_nao.value_counts()
len_nao = len(num_nao)

teste = split_tweet(limpar_caracteres(nubank_teste, 'Teste'), 'Teste')
teste = teste.join(nubank_teste['Avaliacao'])

tudo = tabela_tudo(split_tweet(limpar_caracteres(nu_treino_sim, 'Treinamento'), 'Treinamento'),split_tweet(limpar_caracteres(nu_treino_nao, 'Treinamento'), 'Treinamento'))
total_palavras = len(tudo['palavras'].value_counts())


C:\Users\rafik\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [87]:
i = 0
teste["Avaliacao_nb"] = pd.Series()
teste["Resultado"] = pd.Series()

for linha in teste["palavras"]:
    p_sim = 1
    p_nao = 1
    p_naive = ''
    for palavra in linha:
        try:
            p_sim *= (num_sim[palavra] + 1)/(len_sim + total_palavras)
            p_nao *= (num_nao[palavra] + 1)/(len_nao + total_palavras)
        except:
            p_sim *= 1/(len_sim + total_palavras)
            p_nao *= 1/(len_nao + total_palavras)
    if p_sim > p_nao:
        p_naive = 'sim'
        teste["Avaliacao_nb"][i] = "sim"
        if teste['Avaliacao'][i] == p_naive:
            teste["Resultado"][i] = "Positivo Verdadeiro"
        elif teste['Avaliacao'][i] != p_naive:
            teste["Resultado"][i] = "Positivo Falso"

    elif p_sim < p_nao:
        p_naive = 'nao'
        teste["Avaliacao_nb"][i] = p_naive
        if teste['Avaliacao'][i] == p_naive:
            teste["Resultado"][i] = "Negativo Verdadeiro"
        elif teste['Avaliacao'][i] != p_naive:
            teste["Resultado"][i] = "Negativo Falso"
    i+=1

C:\Users\rafik\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rafik\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\rafik\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [88]:
teste.head(50)

,palavras,Avaliacao,Avaliacao_nb,Resultado
0,"[queria, saber, pq, digio, e, nubank, investem...",sim,nao,Negativo Falso
1,"[4, 7, ainda, nao, temos, previsao, mas, os, n...",nao,nao,Negativo Verdadeiro
2,"[opa, fica, tranquilo, !, chama, a, gente, la,...",nao,nao,Negativo Verdadeiro
3,"[chegou, meu, @nubank, agora, sounu, 💜]",nao,nao,Negativo Verdadeiro
4,"[1, imagina, !, 💜, que, bom, que, gostou, 😉, 9...",nao,nao,Negativo Verdadeiro
5,"[@nubank, pelo, app, ando, mal, sucedido, tamb...",nao,nao,Negativo Verdadeiro
6,"[@nubank, berro, com, o, nome, da, playlist]",nao,nao,Negativo Verdadeiro
7,"[nada, como, ter, que, ligar, 1, 7, dezessete,...",sim,nao,Negativo Falso
8,"[nao, to, conseguindo, pagar, minha, fatura, d...",sim,sim,Positivo Verdadeiro
9,"[no, app, vc, pode, simular, quanto, vai, ter,...",nao,nao,Negativo Verdadeiro


In [89]:
print(teste["Avaliacao"].value_counts(normalize=True)*100)
print('\n')
print(teste["Resultado"].value_counts(normalize=True)*100)

nao    77.0
sim    23.0
Name: Avaliacao, dtype: float64


Negativo Verdadeiro    69.5
Negativo Falso         18.0
Positivo Falso          7.5
Positivo Verdadeiro     5.0
Name: Resultado, dtype: float64


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


A partir dos resultados obtidos do classificador Naive Bayes, observou-se uma grande porcentagem de Negativos Falsos(18%) e uma baixa porcentagem de Positivos Verdadeiros(5%), comparando com a base de testes. Isso occoreu devido ao fato de que a base de treinamento utilizada não possui tweets suficientes para uma análise consistente e há, também, problemas e inversões de sintaxe que o programa não consegue detectar, como por exemplo as duplas negações. Já os Negativos Verdadeiros obtiveram uma boa performance (de 75% para 68.5%),indicando que o classificador possui uma maior precisão ao analizar os tweets irrelevantes. Uma possīvel causa para esse comportamento é de que os tweets relevantes tem maior diversidade de conteúdo que os irrelevantes, visto que os não revelantes se encaixam em basicamente em 3 categorias: conteúdo impróprio, perguntas e citações irrelevantes ao nubank. Essa diferença faz com que a base de dados das palavras irrelevantes do classificador seja mais consistente, ou seja, com menos variação de palavras.

A respeito dos tweets com dupla negação, pode-se dizer que: as mensagens de dupla negação e sarcasmo são interpretadas, na maioria das vezes, como tweets irrelevantes, visto que, para a nossa base de treinamento, os tweets com conotação negativa em sua maior parte são irrelevantes. Portanto, a dupla negação em um tweet classificado como relevante aumenta a P(Tweet|Irelevante), de acordo com o método do Naive Bayes, o que o classifica como irrelevante. Isso explica uma parte da grande quantidade de Negativos Falsos obtidos.

O projeto "Classificador Automático de Sentimento" deve continuar sendo financiado devido a sua boa porcentagem de acertos de Negativos Verdadeiros (89%), com apenas uma pequena base de dados. Isso irá ajudar no filtro para responder os tweets, eliminando a necessidade de um funcionário que analisa os tweets. Dessa forma, a empresa apenas precisará de um funcionário para responder os tweets que já foram pré selecionados pelo classificador. Alguns planos de expansão para o projeto serão: o aumento da base de dados, identificar duplas negações, adicionar palavras semelhantes a certos grupos de relevância, remover artigos e preposições e obter mais níveis de classificação, como: Muito Relevante, Pouco Relevante, Neutro, Pouco Irrelevante, Muito Irrelevante. Essas melhorias irão proporcionar uma maior precisão e uma maior porcentagem de Positivos Verdadeiros.

O classificador não pode ser realimentado pelos tweets classificados pois, loren ipsum dolor sit

Durante o percorrer deste projeto, o classificador de sentimento com Naive Bayes provou-se extremamente útil. Podemos imaginar diferentes cenarios onde ele pode ser aplicado, a fim de facilitar, agiliazar e otimizar processos, como por exemplo: Em um app de streaming de música, podemos prever qual seria a avaliação de uma música a ser lançada, com base nas análises feitas por usuarios e críticos dos metadados da música.